# Idea

Monitoring popular russian IT resurce and monitoring any feadbacks about company

# Imports and installs

In [1]:
%pip install beautifulsoup4
%pip install html5lib
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
import json
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup

# Functions 

# Data processing

In [3]:
url = "https://vc.ru/claim"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
article_links = []
for tag in soup.find_all("a"):
    class_names = tag.get("class", [])
    if "content-link" in class_names:
        article_links.append(tag["href"])

storage = defaultdict(dict)
for link in article_links:
    storage[link]["link"] = link
    response = requests.get(link)
    article = BeautifulSoup(response.text, 'html.parser')
    for tag in article.find_all("h1"):
        storage[link]["name"] = tag.text
    text = ""
    for tag in article.find_all("div"):
        class_names = tag.get("class", []) 
        if "content--full" in class_names:
            text += tag.text
            print
    storage[link]["text"] = text

In [5]:
pd.DataFrame(storage.values()).dropna().sample(5)

,link,name,text
7,https://vc.ru/claim/362344-avito-dostavka-ili-...,\n Авито-Доставка — или как...,\n\nИстория началась в Перми в начале января 2...
6,https://vc.ru/claim/362951-oshibki-kartochek-v...,\n Ошибки карточек в Яндекс...,\n\n\n Продолжение ситуации с ошибочной...
3,https://vc.ru/claim/362550-tinkoff-po-oshibke-...,\n Тинькофф по ошибке удерж...,\n\nВсем привет! Заголовок получился громким и...
8,https://vc.ru/claim/362218-podozrevayu-chto-al...,"\n Подозреваю, что Альфа-ба...",\n\nВсем привет. Буду краток. Являюсь клиентом...
12,https://vc.ru/claim/362027-prishel-razbityy-gr...,\n Пришел разбитый графичес...,\n\n\n Вместо предисловия… \n\n\n\n\...


# Find companies

In [6]:
companies = ["Яндекс", "Yandex", "Apple", "Ozon", "Озон"]
for link, article in storage.items():
    mentiond = []
    for company in companies:
        if company.lower() in article.get("name", "").lower():
            mentiond.append(company)
    
    storage[link]["company"] = mentiond


In [7]:
pd.DataFrame(storage.values()).dropna().sample(5)

,link,name,text,company
10,https://vc.ru/claim/362844-postoyannye-problem...,\n Постоянные проблемы с пр...,\n\n#жалобаozon #ozon #озон #fbo\n\n\nПрошу пр...,[Ozon]
12,https://vc.ru/claim/362027-prishel-razbityy-gr...,\n Пришел разбитый графичес...,\n\n\n Вместо предисловия… \n\n\n\n\...,[]
5,https://vc.ru/claim/362766-avito-ignoriruet-vo...,\n Авито игнорирует вопросы...,"\n\nВсем привет. Не думал, что моей первой пуб...",[]
9,https://vc.ru/claim/362853-vopros-k-sdek,\n Вопрос к СДЭК,\n\n\n\n\n\n\n\n\n\n\nУ меня пара вопросов к c...,[]
1,https://vc.ru/claim/362954-sberbank-uderzhivae...,\n Сбербанк удерживает мои ...,\n\nНесколько лет сижу читаю статьи как разные...,[]


# Save results

In [10]:
fname = "articles.json"
data = [article for article in storage.values()]
with open(fname, "w") as file:
    json.dump(data, file)